In [16]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import os

In [74]:
def load_images():
    path_train_b = "data/train/benign"
    path_train_m ="data/train/malignant"
    path_test_b = "data/test/benign"
    path_test_m = "data/test/malignant"
    paths = [path_train_b, path_train_m , path_test_b, path_test_m]
    
    x_train_b = []
    x_train_m = []
    x_test_b = []
    x_test_m = []
    x = [x_train_b, x_train_m, x_test_b, x_test_m]
    
    for i in range(len(paths)):
        for filename in os.listdir(paths[i]):
            img = cv2.imread(os.path.join(paths[i],filename))
            if img is not None:
                x[i].append(img)
    
    [x_train_b,x_train_m,x_test_b,x_test_m] = x
    return x_train_b,x_train_m,x_test_b,x_test_m
            


In [75]:
x_train_b,x_train_m,x_test_b,x_test_m = load_images()
x_train_b = np.array(x_train_b)
x_train_m = np.array(x_train_m)
x_test_b = np.array(x_test_b)
x_test_m = np.array(x_test_m)

In [76]:
#labels
y_train_b = np.zeros((x_train_b.shape[0],1), dtype = int)
y_train_m = np.ones((x_train_m.shape[0],1), dtype= int)
y_test_b = np.zeros((x_test_b.shape[0],1), dtype = int)
y_test_m = np.ones((x_test_m.shape[0],1), dtype = int)

#merging
x_train = np.concatenate((x_train_b,x_train_m), axis = 0)
y_train = np.concatenate((y_train_b,y_train_m), axis = 0)
x_test = np.concatenate((x_test_b,x_test_m), axis = 0)
y_test = np.concatenate((y_test_b,y_test_m), axis = 0)

X = np.concatenate((x_train,x_test), axis =0)
Y = np.concatenate((y_train, y_test), axis = 0)


In [77]:

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, shuffle = True, random_state = 42)



#shuffle
# r1 = np.arange(x_train.shape[0])
# r2 = np.arange(x_test.shape[0])
# np.random.shuffle(r1)
# np.random.shuffle(r2)
# x_train = x_train[r1]
# y_train= y_train[r1]
# x_test = x_test[r2]
# y_test = y_test[r2]

In [78]:
#reshaping
X_train = X_train.reshape(X_train.shape[0], -1).T
X_test = X_test.reshape(X_test.shape[0], -1).T
Y_train = Y_train.T
Y_test = Y_test.T

print(X_train.shape)
print(Y_train.shape)

(150528, 2637)
(1, 2637)


In [79]:
#normalize
X_train = X_train/255
X_test = X_test/255

In [80]:
def sigmoid(z):
    return 1/(1+ np.exp(-z))


In [81]:
def initialize(size):
    w = np.zeros(shape=(size,1))
    b = 0
    return w, b

In [82]:
def propagation(x,y,w,b):
    
    n = x.shape[1]
    
    #forward prop
    a = sigmoid(w.T @ x + b)
    L = np.square(np.subtract(a,y)).mean()
#     L = np.squeeze(L)
    
    #backward
    dw = 1/n * x @ (a-y).T
    db = 1/n * np.sum(a-y)
    
    return L, dw, db
    

In [83]:
def learning(x,y,w,b, iterations, learning_rate):
    
    Ls = []
    
    for i in range(iterations):
        L, dw, db = propagation(x,y,w,b)
        w = w- learning_rate * dw
        b = b - learning_rate * db
        
        if i % 100 == 0:
            Ls.append(L)
            print(f'Loss after iteration {i}: {L}')
    
    return w,b,dw,db,Ls
        

In [84]:
def predict(x,y,w,b):
    y_hat = np.zeros((1,x.shape[1]))
    w  = w.reshape(x.shape[0],1)
    a = sigmoid(w.T @ x + b)
    
    for i in range(a.shape[1]):
        if a[0,i] >0.5:
            y_hat[0,i] = 1
        else:
            y_hat[0,i] = 0
    
    print("Accuracy: " + str(100*np.sum((y_hat==y)/x.shape[1]))+ "%")
    
    return y_hat

In [85]:
def model(xtrain, ytrain, xtest, ytest, iterations, learning_rate):
    w, b = initialize(xtrain.shape[0])
    w,b,dw,db,Ls = learning(xtrain, ytrain, w, b, iterations, learning_rate)
    y_hat_train = predict(xtrain,ytrain, w, b)
    y_hat_test = predict(xtest,ytest,w,b)

    return Ls
    

In [ ]:
iterations = 100
learning_rate = 0.005
L = model(X_train,Y_train, X_test, Y_test,iterations, learning_rate)
plt.plot(L)
plt.ylabel('loss')
plt.xlabel('iterations (per tens)')
plt.title("Learning rate =" + str(learning_rate))
plt.show()